__Falcon 9 Data Collection_ Web Scraping__  

SpaceX is a well-known private company famous for several historic milestones in launching rockets and return the first stage from the low earth orbit. This capability enabled SpaceX to advertise the Falcon 9 rocket with the price of $62 million which is more than 60% cheaper that the other competitive companies.  

The present notebook extracts data from [Wikipedia page](https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922) and performs exploratory data analysis using SQL queries. 

The present lab is a part of [Applied Data Science Capstone Course](https://www.coursera.org/learn/applied-data-science-capstone) offered by [Coursera.org](https://www.Coursera.org)

![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_1\_L2/images/Falcon9\_rocket_family.svg)

### List of libraries

In [16]:
import requests
import pandas as pd
import numpy as np
import datetime
import os
from bs4 import BeautifulSoup
import unicodedata
# below libraries will be used to submit query the database in IBM DB2
import sqlalchemy
import ibm_db_sa
import ibm_db

In [25]:
# changing the current directory
os.chdir('G:\My Drive\Career\IBM certificate\Database\Spacex')

### Data Collection

In [6]:
# let's request the launch wiki page from its url to extract the list of 
#Falcon 9 and Falcon Heavy launches published on June 2021
url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
response = requests.get(url)

In [8]:
soup = BeautifulSoup(response.text,'html.parser')

### Extracting columns names

In [9]:
html_tables = soup.find_all('table')

In [11]:
# let's use table number 3 to extract columns names
first_launch_table = html_tables[2]

In [12]:
column_names = []
headers = first_launch_table.find_all('th')
for th in headers:
    if th.string:
        column_names.append(th.text.strip())
    elif '[c]' in th.text:
        column_names.append(th.text.split('[')[0])
    else:
        column_names.append(((list(th.strings)[0].split(',')[0]+' '+list(th.strings)[1].split()[0]).split('[')[0]))
column_names[:10]

['Flight No.',
 'Date and time',
 'Version Booster',
 'Launch site',
 'Payload',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launch outcome',
 'Booster landing']

### Parsing the data from launch HTML table

In [14]:
# let's create a dictionary of from the list above with some modifications
launch_dict = dict.fromkeys(column_names[:10])
launch_dict
del launch_dict['Date and time']

launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
launch_dict['Version Booster'] = []
launch_dict['Booster landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []
#let's fill up the dictionary above
for table_number, table in enumerate(soup.find_all("table","wikitable plainrowheaders collapsible")):
    for rows in table.find_all('tr'):
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag=False
        # get table element
        row = rows.find_all('td')
        # if it is a number save cells in a dictionary
        if flag:
            # Flight_number
            launch_dict['Flight No.'].append(flight_number)
            # Date
            launch_dict['Date'].append(list(row[0].strings)[0].strip(','))
            # Time
            launch_dict['Time'].append(list(row[0].strings)[1].strip('\n'))
            # Version Booster
            l_row1 = list(row[1].strings)
            bv = [(i.strip('\n')) for index,i in enumerate(l_row1) if list(l_row1[index])[0]=='B']
            if bv:
                launch_dict['Version Booster'].append(l_row1[0].strip('♺')+' '+bv[0])
            else:
                launch_dict['Version Booster'].append(l_row1[0].strip('♺'))
            # Launch site
            if row[2].a.string:
                launch_dict['Launch site'].append(row[2].a.string.strip('\n'))
            else:
                launch_dict['Launch site'].append(np.nan)
            # Payload
            launch_dict['Payload'].append(row[3].a.string)
            # Payload mass
            if row[4].text.strip():
                l_row4 = list(row[4].text.strip())
                launch_dict['Payload mass'].append((row[4].text.strip().split('kg')[0].strip('~')).replace(',',''))
            else:
                launch_dict['Payload mass'].append(np.nan)
            # Orbit
            launch_dict['Orbit'].append(row[5].a.string)
            # Customer
            if row[6].a:
                if row[6].a.string:
                    customer = row[6].a.string
            else:
                customer = "unknown"
            launch_dict['Customer'].append(customer)
            # Launch outcome
            launch_outcome = list(row[7].strings)[0].strip('\n')
            launch_dict['Launch outcome'].append(launch_outcome)
            # Booster landing
            launch_dict['Booster landing'].append(list(row[8].strings)[0].strip())

### Creating a dataframe

In [15]:
df_launch = pd.DataFrame(launch_dict)
df_launch.head()

,Flight No.,Version Booster,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Booster landing,Date,Time
0,1,F9 v1.0 B0003.1,CCAFS,Dragon Spacecraft Qualification Unit,NaN,LEO,SpaceX,Success,Failure,4 June 2010,18:45
1,2,F9 v1.0 B0004.1,CCAFS,Dragon,NaN,LEO,NASA,Success,Failure,8 December 2010,15:43
2,3,F9 v1.0 B0005.1,CCAFS,Dragon,525,LEO,NASA,Success,No attempt,22 May 2012,07:44
3,4,F9 v1.0 B0006.1,CCAFS,SpaceX CRS-1,4700,LEO,NASA,Success,No attempt,8 October 2012,00:35
4,5,F9 v1.0 B0007.1,CCAFS,SpaceX CRS-2,4877,LEO,NASA,Success,No attempt,1 March 2013,15:10


In [24]:
# let's download the dataset
df_launch.to_csv('SPACEX_TABLE01.csv')